In [2]:
import sqlite3
import pandas as pd

# Create a connection
conn = sqlite3.connect('timeseries.db')

# Create a cursor
cursor = conn.cursor()

# Create a table
cursor.execute('CREATE TABLE TimeSeries (Name VARCHAR(255), Start DATETIME, End DATETIME, Activity VARCHAR(255));')

# Insert values
with open('timeseries.csv', 'r') as f:
    df = pd.read_csv(f)
    df.to_sql('TimeSeries', conn, if_exists='append', index=False)

conn.commit()


In [8]:
cursor.execute('SELECT * FROM TimeSeries;')
ts = cursor.fetchall()
ts_df = pd.DataFrame(ts, columns=[column[0] for column in cursor.description])
conn.commit()
ts_df.head()

,Name,Start,End,Activity
0,Priyanka,10/Oct/23 10:43 PM,20/Jul/23 03:31 AM,Inspection
1,Jyoti,24/Aug/23 05:55 AM,17/May/23 08:19 PM,Remote Inspection
2,Jyoti,08/Jun/23 08:19 AM,08/Apr/23 05:55 AM,Updates
3,Priyanka,21/Sep/23 03:31 PM,27/May/23 10:43 AM,Reporting
4,Priyanka,07/Oct/23 03:31 AM,30/Apr/23 01:07 PM,Reply to Customers


In [10]:
cursor.execute('''
    SELECT
        t1.Name AS 'Bot Name',
        t1.Start AS 'Start',
        t2.End AS 'End',
        GROUP_CONCAT(t1.Activity, ', ') AS 'Activities'
    FROM
        TimeSeries t1
    JOIN
        TimeSeries t2 ON t1.Name = t2.Name AND t1.Start <= t2.End
    GROUP BY
        t1.Name, t1.Start, t2.End
    ORDER BY
        t1.Name, t1.Start, t2.End;
    ''')
query = cursor.fetchall()
query_df = pd.DataFrame(query, columns=[column[0] for column in cursor.description])
query_df.head()

,Bot Name,Start,End,Activities
0,Deepti,01/Apr/23 01:07 AM,01/Apr/23 01:07 AM,"Business Development, Business Development, Bu..."
1,Deepti,01/Apr/23 01:07 AM,01/Apr/23 01:07 PM,"Business Development, Business Development, Fu..."
2,Deepti,01/Apr/23 01:07 AM,01/Apr/23 03:31 AM,"Business Development, Business Development, Fu..."
3,Deepti,01/Apr/23 01:07 AM,01/Apr/23 03:31 PM,"Business Development, Fund raising"
4,Deepti,01/Apr/23 01:07 AM,01/Apr/23 05:55 AM,"Business Development, Fund raising"


In [9]:
conn.close()